In [2]:
import sys

sys.path.append("..")

In [122]:
import pandas as pd

from tqdm import tqdm
from functools import reduce
from collections import Counter
from classifier.preprocessor import Preprocessor

In [4]:
tqdm.pandas()
pd.set_option("max_colwidth", None)

In [5]:
df = pd.read_csv("../data/train.csv")

In [6]:
preprocessor = Preprocessor(
    do_stem=False,
    remove_stop_words=True, 
    black_list=r"\b.*id.*\b",
    remove_single_letter_token=True,
    max_token_length=25,
)

In [7]:
df["text_preprocessed"] = df["text"].progress_apply(preprocessor._preprocess)

100% 38740/38740 [00:14<00:00, 2638.94it/s]


In [8]:
df[["text", "text_preprocessed"]].sample(10)

,text,text_preprocessed
21170,Первенство области среди юниоров и юниорок до 20 лет 15 16 июня стадион СК Урал Открыт прием заявок С целью подготовки к первенствам России на этот старт будут допущены спортсмены U18 а также в к старшие возрастные категории. Программа предварительная как всегда будет актуализирована после окончания приема заявок.,первенство области среди юниоров юниорок лет июня стадион ск урал открыт прием заявок целью подготовки первенствам россии старт будут допущены спортсмены u18 также старшие возрастные категории программа предварительная актуализирована окончания приема заявок
38510,Льюис Хэмилтон Ferrari и Red Btokenoid выступают в своей лиге На квалификации в Бахрейне в Mertokenoid удалось прибавить после неудачных тренировок но если бы Джордж Расселл не потерял несколько десятых на первом секторе результаты могли быть лучше. Льюис Хэмилтон 5 й Не могу сказать что чувствую облегчение – я очень доволен квалификацией после того какие позиции мы занимали вчера и сколько проблем с машиной у нас возникло – пилотировать ее было настоящим кошмаром. Но мы сосредоточились на работе и я горжусь всеми в команде – они сохраняли позитивный настрой а в итоге я квалифицировался пятым. Ferrari и Red Btokenoid Ratokenoid выступают в своей лиге но я доволен своим результатом. Это не первый ряд стартовой решетки но мы полностью выложимся в гонке. Завтра машины Red Btokenoid Ratokenoid и Ferrari быстро уедут вперед – мы им не соперники. Вчера их гоночный темп был на секунду лучше нашего. Скорее всего мне предстоит бороться с теми кто стартует позади. Конечно я попытаюсь догнать лидеров но как я сказал они намного быстрее нас. Джордж Расселл 9 й Я доволен своим выступлением во второй части квалификации. В финале я решил иначе работать с резиной и атаковал на прогревочном круге а в результате в первом повороте у меня не было сцепления. Я надеялся отыграть десятую а в итоге уступил секунду своему результату во второй сессии. Жаль что так получилось. Мы рассчитываем на другие результаты и машина позволяет добиться большего чем девятое место. Постараемся отыграться и навязать борьбу Ferrari и Red Btokenoid Ratokenoid. Не лучший результат но гонка только завтра. Мы делаем один шаг вперед и два назад команда прогрессирует но затем проблемы возвращаются. Все это сложно ведь приходится решать основные проблемы а не разбираться в деталях и все настраивать. Но все так как есть. Мы знаем что скоро вновь сможем бороться за победы – и делаем для этого все возможное. F1 Mertokenoid Hamtokenoid Rtokenoid Bahratokenoid,льюис хэмилтон fеrrаri rеd выступают своей лиге квалификации бахрейне удалось прибавить неудачных тренировок джордж расселл потерял несколько десятых первом секторе результаты могли льюис хэмилтон могу сказать чувствую облегчение очень доволен квалификацией какие позиции занимали вчера сколько проблем машиной возникло пилотировать настоящим кошмаром сосредоточились работе горжусь всеми команде сохраняли позитивный настрой итоге квалифицировался пятым fеrrаri rеd выступают своей лиге доволен своим результатом это первый ряд стартовой решетки полностью выложимся гонке завтра машины rеd fеrrаri быстро уедут вперед соперники вчера гоночный темп секунду нашего скорее предстоит бороться теми стартует позади попытаюсь догнать лидеров сказал намного быстрее джордж расселл доволен своим выступлением второй части квалификации финале решил иначе работать резиной атаковал прогревочном круге результате первом повороте сцепления надеялся отыграть десятую итоге уступил секунду своему результату второй сессии жаль получилось рассчитываем другие результаты машина позволяет добиться большего девятое место постараемся отыграться навязать борьбу fеrrаri rеd лучший результат гонка завтра делаем шаг вперед назад команда прогрессирует затем проблемы возвращаются это сложно приходится решать основные проблемы разбираться деталях настраивать знаем скоро вновь сможем бороться победы делаем возможное ф1
16994,weareatokenoid Котировки букмекеров на побе

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 5), max_features=10_000)

In [11]:
vectorizer.fit(df.text_preprocessed)

TfidfVectorizer(analyzer='char_wb', max_features=10000, ngram_range=(2, 5))

In [12]:
len(vectorizer.vocabulary_)

10000

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
model = LogisticRegression()

In [15]:
model.fit(
    vectorizer.transform(df.text_preprocessed),
    df.category
)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

## Submit

In [16]:
hidden = pd.read_csv("../data/test.csv")

In [17]:
hidden["text_preprocessed"] = hidden["text"].progress_apply(preprocessor._preprocess)

100% 26260/26260 [00:10<00:00, 2623.68it/s]


In [18]:
hidden_features = vectorizer.transform(hidden.text_preprocessed.tolist())

In [19]:
hidden_predictions = model.predict_proba(hidden_features)

In [20]:
hidden["predictions"] = list(hidden_predictions)

In [21]:
hidden["category"] = hidden["predictions"].apply(lambda predictions: model.classes_[predictions.argmax()])

In [22]:
hidden["score"] = hidden["predictions"].apply(max)

In [28]:
hidden

,oid,text,text_preprocessed,predictions,category,score
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПКОЙ ПИШИТЕ В ЛС 33 ВСЕ ГАРАНТИИ С МЕНЯ 33,спочно сообщество продается 1300р покупкой пишите лс гарантии,"[0.035579374846412104, 0.0851730316527113, 0.04129502931453788, 0.017797492338485307, 0.6124008670220886, 0.0510488987985959, 0.046494972440804985, 0.006985568385313507, 0.037595040440426726, 0.029061563852498633, 0.011759796462441153, 0.018372645367180678, 0.006435719078503139]",esport,0.612401
1,452466036,Естественное восстановление после тяжелой тренировки занимает 72 часа. С криокамерой 24 часа. Быстрее в 3 раза из интервью крио физиотерапевта на ютуб канале Khamzat Chtokenoid Заход в криокамеру с температурой минус 87 С равноценен 20 25 минутам восстановления в ледяной ванне. Криокамера ускоряет твое восстановление на 300. При естественном восстановлении организму после тяжелой тренировки требуется обычно 72 часа. Криокамера даст вам полное восстановление за 24 часа нвк,естественное восстановление тяжелой тренировки занимает часа криокамерой часа быстрее раза интервью крио физиотерапевта ютуб канале кhаmzаt заход криокамеру температурой минус равноценен минутам восстановления ледяной ванне криокамера ускоряет твое восстановление естественном восстановлении организму тяжелой тренировки требуется обычно часа криокамера даст полное восстановление часа нвк,"[0.053742274465861536, 0.03524614246101791, 0.05360138170133902, 0.028002339133281665, 0.0867504462501257, 0.08957698998147577, 0.08338951451674577, 0.027836471279993988, 0.2032343865256335, 0.08717929270784541, 0.02753467534997278, 0.02966623475396306, 0.19423985087274379]",martial_arts,0.203234
2,161038103,Тема нарядов продолжается Одна из британских журналисток захейтела образ Марии Саккари на Уимблдоне. Наряды действительно подчеркивают индивидуальность того кто их носит. Тот же наряд который выглядел сексуально на Младенович выглядит свирепо на Саккари,тема нарядов продолжается одна британских журналисток захейтела образ марии саккари уимблдоне наряды действительно подчеркивают индивидуальность носит наряд который выглядел сексуально младенович выглядит свирепо саккари,"[0.012931483449979303, 0.07413236684436951, 0.05417488775369658, 0.03640020924566133, 0.0324574880863148, 0.09298603849139302, 0.15020416043792179, 0.08950172793575503, 0.0966564955506647, 0.053917758170118, 0.2547469876477469, 0.027226299740634435, 0.024664096645744652]",tennis,0.254747
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здесь происходит? Ответ скоро узнаешь. Ты получишь ряд подсказок каждая из которых дополняет общую картину. В конце самый внимательный заберет награду – одну из ТРЕХ Ptokenoid 5 и много других призов. Награду получат те кто разгадают каждую подсказку а в конце конкурса максимально подробно ответят на вопрос чему посвящен этот паблик. Правильные ответы на все подсказки раскроем в самом конце. Готов? Жми колокольчик следи за таймером и не упусти шанс стать частью События. ДЛЯ УЧАСТИЯ Подпишись на этот паблик Лайкни этот пост и поделись им у себя на странице Выбери ответ который кажется тебе наиболее верным.,привет избранный спрашиваешь происходит ответ скоро узнаешь получишь ряд подсказок каждая которых дополняет общую картину конце самый внимательный заберет награду одну трех других призов награду получат те разгадают каждую подсказку конце конкурса максимально подробно ответят вопрос чему посвящен паблик правильные ответы подсказки раскроем самом конце готов жми колокольчик следи таймером упусти шанс стать частью события участия подпишись паблик лайкни пост поделись странице выбери ответ который кажется тебе наиболее верным,"[0.02823392257214016, 0.04242473584615216, 0.01592939373057326, 0.22526114293662045, 0.16493005979535436, 0.2661660164497747, 0.007154043702590915, 0.0112393082777279, 0.02352794837435876, 0.056964344452399576, 0.011608959716931776, 0.1325570028223918, 0.014003121322984225]",extreme,0.266166
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Ан

In [29]:
hidden.groupby("oid").apply(lambda df: Counter(df.category))

oid
1622114                                     {'esport': 5, 'athletics': 3, 'boardgames': 1, 'extreme': 1}
1663817               {'autosport': 6, 'winter_sport': 1, 'extreme': 1, 'boardgames': 1, 'motosport': 1}
3174332                                 {'football': 2, 'martial_arts': 2, 'basketball': 5, 'hockey': 1}
3469228                   {'extreme': 4, 'martial_arts': 3, 'esport': 1, 'boardgames': 1, 'football': 1}
3905302                                                {'boardgames': 6, 'esport': 2, 'winter_sport': 2}
                                                        ...                                             
998309713    {'esport': 5, 'basketball': 1, 'football': 1, 'hockey': 1, 'extreme': 1, 'martial_arts': 1}
998565619                                                                   {'football': 2, 'esport': 8}
999112505                                                                             {'boardgames': 10}
999361308                                          

In [139]:
mask = hidden.score >= 0.4


dfs = [
    hidden[mask].groupby(["oid"]).apply(
        lambda df: Counter(df.category).most_common(1)[0][0]
    ).rename("popular_category").reset_index(),
    
    hidden[mask].groupby(["oid"]).apply(
        lambda df: Counter(df.category).most_common(1)[0][1]
    ).rename("popular_category_count").reset_index(),
    
    hidden[mask].groupby(["oid"]).apply(
        len
    ).rename("posts_count").reset_index(),
]

result = reduce(lambda left, right: pd.merge(left, right, on='oid'), dfs)

result = result[result.popular_category_count / result.posts_count >= 0.3]

result

,oid,popular_category,popular_category_count,posts_count
0,1622114,esport,3,5
1,1663817,autosport,4,6
2,3174332,basketball,5,5
3,3469228,extreme,1,3
4,3905302,boardgames,6,6
...,...,...,...,...
2608,998309713,esport,4,4
2609,998565619,esport,7,8
2610,999112505,boardgames,10,10
2611,999361308,tennis,7,7


In [133]:
result[["oid", "popular_category"]].rename(columns={"popular_category": "category"}).to_csv("../data/submission.csv", index=False)

In [82]:
hidden.to_csv("../data/submission_log_reg.csv", index=False)

In [23]:
hidden[hidden.score >= 0.8][["oid", "category"]].to_csv("../data/submission.csv", index=False)